<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.8-7.21/lab11-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab11-3 - RNN Long Sequence
===
기존의 ```hihello```와 ```charseq```와 어떤 부분이 다를까?
## 목차
1. Longseq introduction
2. 긴 문장으로부터 sequence dataset 만들기
3. Adding FC layer and stacking RNN
4. Code run through

# 1. Longseq introduction
아주 긴 문장이 있으면 특정 사이즈로 잘라서 쓴다.

# 2. 긴 문장으로부터 sequence dataset 만들기
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FNa2m9%2Fbtq0MT8SlQj%2FodFKBKQCGkAyVz80Qujsz0%2Fimg.png)    

특정 크기만큼의 윈도우가 있다. 위 사진에서의 윈도우는 10칸이다. 이 윈도우를 오른쪽으로 한 칸씩 이동한다. 


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [9]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [10]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [13]:
# data setting
x_data = []
y_data = []
sequence_length = 10

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i: i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    x_data.append([char_dic[c] for c in x_str])     # x str to index
    y_data.append([char_dic[c] for c in y_str])     # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [14]:
# Adding FC Layer and stacking RNN

# declare RNN + FC
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2)

In [18]:
# loss & optimizer setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)         # 확률이 가장 큰 prediction 결과값들을 불러온다.
    predict_str = ""
    for j, result in enumerate(results):
        print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
70 100 s and work 0.257905513048172
70 101 iand work, 0.257905513048172
70 102 tnd work,  0.257905513048172
70 103 nd work, b 0.257905513048172
70 104 d work, bu 0.257905513048172
70 105  aork, but 0.257905513048172
70 106 tork, but  0.257905513048172
70 107 ork, but r 0.257905513048172
70 108 nk, but ra 0.257905513048172
70 109  , but rat 0.257905513048172
70 110 s but rath 0.257905513048172
70 111  dut rathe 0.257905513048172
70 112 tut rather 0.257905513048172
70 113 ut rather  0.257905513048172
70 114 t rather t 0.257905513048172
70 115  dather te 0.257905513048172
70 116 tather tea 0.257905513048172
70 117  ther teac 0.257905513048172
70 118 nher teach 0.257905513048172
70 119  em toach  0.257905513048172
70 120 er toach t 0.257905513048172
70 121 r toach th 0.257905513048172
70 122  teach the 0.257905513048172
70 123 toach them 0.257905513048172
70 124  ach them  0.257905513048172
70 125 rch them t 0.257905513048172
70 126 nh them to 0.25790551